In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import pyplot as plt
from numpy import random as rn
from scipy.stats import norm, kurtosis, skew
from numpy.random import seed
from numpy.random import rand
import scipy
import math



In [ ]:
#land distribution cost/acre (2022) - > Inflated to 2023 prices
lcost = np.array([4580, 3400, 6600, 3960, 3800, 3340, 3550, 3410, 3000, 2560, 2250, 2650, 1390, 4200, 1770, 3700, 1030, 1060, 610, 2810, 850, 7040, 12000, 3040, 3100, 6490, 13700, 9800, 2860, 9700, 15200, 5350, 1540, 3450, 7350, 17500, 4200, 5960, 5850, 6150, 5700, 7560, 8900, 8000, 9400, 4150, 7200, 2780, 2630, 3750, 2050, 2600, 4620, 4350, 5150, 4700, 5100, 3000])
lmin = np.min(lcost)*640 * (302.9/292.7)
lmax = np.max(lcost) *640 * (302.9/292.7)


#structure/facilities cost (must inflate to 2022 $) -> (2023 CPI/Old CPI)
nif_sc = 270000000*(302.9/179.9) #2002
omega_sc = 21000000 * (302.9/152.4) #1995

#reaction vessel cost (1994)
tc_low = 15600000 * (302.9/148.2)
tc_high = 45500000 * (302.9/148.2)

#steam turbine equipment cost (2010)
sc_low = 45000000*2 * (302.9/218.1)
sc_high = 45000000*4 * (302.9/218.1)

#beta factor ($/j)
b_high = 10
b_low = 1.4


data = np.array([[b_low, lmin, tc_low, omega_sc, sc_low],  [b_high, lmax, tc_high, nif_sc, sc_high]])

In [ ]:
# Creates pandas DataFrame. 
columns = ['beta', 'land cost', "target area", 'structure cost', 'thermal cycle']
df = pd.DataFrame(data, index =['Minimum','Maximum'], columns = columns ) 
df

In [ ]:
# Rename columns
df.rename_axis("cost object", inplace=True)

# Add a new 'Total' column with row-wise sums
df['Total'] = df.sum(axis=1)

# Transpose the DataFrame
df_transposed = df.T

# Display the transposed DataFrame
print(df_transposed)

In [ ]:
#MONTE CARLO SIMULATION
# seed random number generator
seed()
N = 879
M = N
#driver energy (J)
E_d = 11000000
# generate random numbers between 0-1
lcost_rand = rand(M)
beta_rand = rand(M)
turbine_rand = rand(M)
struc_rand = rand(M)
target_rand = rand(M)

land = 1*np.ones((M,1))
Beta = 1*np.ones((M,1))
turbine = 1*np.ones((M,1))
structure = 1*np.ones((M,1))
target = 1*np.ones((M,1))

for i in range(0,M):
    land[i]=lcost_rand[i]*(lmax-lmin)+lmin
    Beta[i]=beta_rand[i]*(b_high-b_low)+b_low
    turbine[i]=turbine_rand[i]*(sc_high-sc_low)+sc_low
    structure[i]=struc_rand[i]*(nif_sc-omega_sc)+omega_sc
    target[i]=target_rand[i]*(tc_high-tc_low)+tc_low

Total = 1*np.ones((M,1))
#Total

for i in range(0,M):
 Total[i]=land[i]+Beta[i]*(E_d)+turbine[i]+structure[i]+target[i]

Total

#simple rounding function 
def round_to_sf(number, sf):
    """
    Round a number to a specified number of significant figures.

    Parameters:
    - number: The number to be rounded.
    - sf: The number of significant figures.

    Returns:
    - The rounded number.
    """
    if number == 0:
        return 0

    # Calculate the order of magnitude
    order_of_magnitude = sf - int(math.floor(math.log10(abs(number)))) - 1

    # Round to the desired number of significant figures
    rounded_number = round(number, order_of_magnitude)

    return rounded_number


Expected_Value1 = round_to_sf(int(np.mean(Total)),3)
print('The expected value of the total cost of the project is: $' , Expected_Value1)

Median1 = round_to_sf(int(np.median(Total)),3)
print(' The median of the total cost of the project is:', '${:,}'.format(Median1))

sigma1 = round_to_sf(int(np.std(Total)),3)
print(' The standard deviation of the total cost of the project is:', '${:,}'.format(sigma1))

stdError1 = round_to_sf(int(np.sqrt((sigma1**2/N))),2)
print(' The standard error of the estimate is:', '${:,}'.format(stdError1))




In [ ]:
#Plotting Overnight Cost of Facility w/E_d = 11MJ

num_bins = 20
xlabel = "Cost $"
fsize = (10, 5)

fig, ax = plt.subplots(figsize=fsize)

n, bins, patches = ax.hist(Total, num_bins, color="royalblue", rwidth=0.9)

ax.set_xlabel(xlabel, fontsize=18)
ax.set_ylabel("Frequency", fontsize=18)

ax.legend([r'$\mu = \$ {:.2f}$, $\sigma = \$ {:.2f}$'.format(int(Expected_Value1), int(sigma1))])



plt.grid()
plt.show()


In [ ]:
# Plot histograms for each parameter
plt.figure(figsize=(15, 8))

plt.subplot(2, 3, 1)
plt.hist(land, bins=30, color='blue', edgecolor='black', alpha=0.7)
plt.title('Distribution of Land Cost')
plt.xlabel('Land Cost')
plt.ylabel('Frequency')

plt.subplot(2, 3, 2)
plt.hist(Beta, bins=30, color='green', edgecolor='black', alpha=0.7)
plt.title('Distribution of Beta')
plt.xlabel('Beta value ($/J)')
plt.ylabel('Frequency')

plt.subplot(2, 3, 3)
plt.hist(turbine, bins=30, color='red', edgecolor='black', alpha=0.7)
plt.title('Distribution of Thermal Cycle Cost')
plt.xlabel('Turbine Cost')
plt.ylabel('Frequency')

plt.subplot(2, 3, 4)
plt.hist(structure, bins=30, color='purple', edgecolor='black', alpha=0.7)
plt.title('Distribution of Structure Cost')
plt.xlabel('Structure Cost')
plt.ylabel('Frequency')

plt.subplot(2, 3, 5)
plt.hist(target, bins=30, color='orange', edgecolor='black', alpha=0.7)
plt.title('Distribution of Target Chamber Cost')
plt.xlabel('Target Cost')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()




In [ ]:
#driver energy dependence
Expected_beta1 = int(np.mean(Beta))
print('The expected value of beta:' , '${:,}'.format(Expected_beta1))

Expected_struc1 = int(np.mean(structure))
Expected_turbine1 = int(np.mean(turbine))
Expected_target1 = int(np.mean(target))
Expected_land1 = int(np.mean(land))

energy = np.linspace(1*10**6, 50*10**6, 100)

def total_func(energy):
    return Expected_struc1 + Expected_land1 + Expected_target1 + Expected_turbine1 + Expected_beta1*energy


# Plot in Logarithmic Scale
plt.semilogx(energy, total_func(energy), label = 'Total project cost ($)')

plt.vlines(11000000, ymin=4.5*1e8, ymax=total_func(10000000), linestyle='--', color='red', label='11MJ Driver cost')

# Customize Plot (if needed)
#plt.title('Total Cost vs. Driver Energy (Logarithmic Scale)')
plt.xlabel('Driver Energy (Joules) - Logarithmic Scale')
plt.ylabel('Total Cost ($)')
plt.grid(True)
plt.legend()
# Show the Plot
plt.show()


In [ ]:
#Generating Capacity calulating + overnight cost

def generating_capacity(E_d, n_d, G, n_th, f):
    E_in = E_d * n_d
    E_fus = G * E_in
    E_grid = (1-f)*n_th*E_fus

    #J/year (assuming a constant run time)
    energy_yr = E_grid * 10 * (60)*(60)*(24)*(365) 
    

    #conversion to kW
    capacity =  energy_yr/((3.6*10**6)*8760)


    return capacity 


#Assumption (1): E_d = 10MJ n_d = 0.2, G = 10, n_th = 0.4, f = 1/4
capacity_1 = generating_capacity(11000000, 0.2, 10, 0.4, 1/4)

overnight_cost1 = round_to_sf(total_func(11000000)/capacity_1, 3)

con_in = 1.96 * stdError1

print('overnight cost 1:', int(overnight_cost1), '$/kWW +/-', int(con_in/capacity_1), '$/kW')

#Assumption (2): E_d = 10MJ n_d = 0.2, G = 50, n_th = 0.4, f = 1/4
capacity_2= generating_capacity(11000000, 0.2, 50, 0.4, 1/4)

overnight_cost2 = round_to_sf(total_func(11000000)/capacity_2, 3)
print('overnight cost 2:', int(overnight_cost2), '$/kWW +/-', int(con_in/capacity_2), '$/kW')

#Assumption (3): E_d = 10MJ n_d = 0.2, G = 100, n_th = 0.4, f = 1/4
capacity_3= generating_capacity(11000000, 0.2, 100, 0.4, 1/4)

overnight_cost3 = round_to_sf(total_func(11000000)/capacity_3, 3)
print('overnight cost 3:', int(overnight_cost3), '$/kWW +/-', int(con_in/capacity_3), '$/kW')






In [ ]:
#future cost calculations

#nominal build time for advanced nuclear T = 4.5 years 

#Empirical build time for advanced nuclear T = 14 years

#inflation rate (current) 3.2%

#interest rate 8%/year (BBB financing)

def interest(x, y, T):
    # Define the integrand function
    integrand = lambda tau: 6/T**3 * tau * (T - tau) * np.exp((x * tau)) * np.exp(y*(T-tau))

    # Perform numerical integration
    result, _ = scipy.integrate.quad(integrand, 0, T)

    return result 


#nominal build time capital costs

#capital cost 1 (G = 10, E_d = 11MJ T = 4.5)
future_cost1 = round_to_sf(overnight_cost1 * interest(0.04, 0.08, 4.5), 3)
print('rate based capital cost (ON1, T_1):', future_cost1, '$/kWW +/-', int(con_in *interest(0.04, 0.08, 4.5) /capacity_1), '$/kW')

#capital cost 2 (G = 50, E_d = 11MJ T = 4.5)
future_cost2 = round_to_sf(overnight_cost2 * interest(0.04, 0.08, 4.5),3)
print('rate based capital cost (ON2, T_1):', future_cost2, '$/kWW +/-', int(con_in *interest(0.04, 0.08, 4.5) /capacity_2), '$/kW')

#capital cost 3 (G = 100, E_d = 11MJ T = 4.5)
future_cost3 = round_to_sf(overnight_cost3 * interest(0.04, 0.08, 4.5),3)
print('rate based capital cost (ON3, T_1):', future_cost3, '$/kWW +/-', int(con_in *interest(0.04, 0.08, 4.5) /capacity_3), '$/kW')


#empirical build time capital costs

#capital cost 1 (G = 10, E_d = 10MJ T = 14)
future_cost1_T2 = round_to_sf(overnight_cost1 * interest(0.04, 0.08, 14),3)
print('rate based capital cost (ON1, T_2):', future_cost1_T2, '$/kWW +/-', int(con_in *interest(0.04, 0.08, 14) /capacity_1), '$/kW')

#capital cost 2 (G = 50, E_d = 10MJ T = 14)
future_cost2_T2 = round_to_sf(overnight_cost2 * interest(0.032, 0.08, 14),3)
print('rate based capital cost (ON2, T_2):', future_cost2_T2, '$/kWW +/-', int(con_in *interest(0.04, 0.08, 14) /capacity_2), '$/kW')

#capital cost 3 (G = 100, E_d = 10MJ T = 14)
future_cost3_T2 = round_to_sf(overnight_cost3 * interest(0.032, 0.08, 14),3)
print('rate based capital cost (ON1, T_2):', future_cost3_T2, '$/kWW +/-', int(con_in *interest(0.04, 0.08, 14) /capacity_3), '$/kW')


#produces rate based capital cost of all 3 cases for two different build times +/- CI